In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install git+https://github.com/rcabanasdepaz/exPandas.git@main
!pip3 install logging-reset


  Cloning https://github.com/rcabanasdepaz/exPandas.git (to revision main) to /tmp/pip-req-build-a0frows3
  Running command git clone -q https://github.com/rcabanasdepaz/exPandas.git /tmp/pip-req-build-a0frows3
  Created wheel for expandas: filename=expandas-0.0.1_SNAPSHOT-py3-none-any.whl size=9973 sha256=22327a86786389d4eb0c36a6313b0f87f3389dff9cbf098e52c8dd1ccd9ab7d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-9nvfrmy5/wheels/7d/01/21/f72241ae34ce9c3b1a967fa4a5c417a07128120b9fe03fa30c
Successfully built expandas


In [2]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import logreset

import conf

import expandas as expd

#from experiments import *

In [3]:
import subprocess
import datetime
import os

import pandas as pd



def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")

exec_bash_print("ls")

GenModels.ipynb
Untitled.ipynb
__pycache__
benckmark_data.ipynb
conf.py
logfile.log
untitled.txt
untitled1.txt


In [4]:
import logging
import logreset
from datetime import datetime


def setup():
    strtime = datetime.now().strftime("%Y%m%d_%H%M%S")
    logreset.reset_logging()
    filename=f'{conf.log_folder}{strtime}.log'
    logging.basicConfig(filename=filename, 
                        format='[%(asctime)s][%(levelname)s][python] %(message)s', 
                        datefmt='%Y-%m-%d_%H:%M:%S',  level=logging.DEBUG)
    
    logging.info(f"Logging info to {filename}")
    
    return filename, strtime

logfile, strtime = setup()


In [5]:
def run_crema(filename:str,  target:int = 0, observed:str ="1", method:str = "cve",
              measure_time:bool = True, runs:int=1, warmups:int=0, timeout:int = 600,
              logfile:str=None, vm_flags = "",
              **kwargs):
        
    observed=str(observed)
    javafile=f"{conf.java_src}RunCrema.java"
    cmd = "-t" if measure_time else ""
    cmd += f" --method={method} -r {int(runs)} -w {int(warmups)} -x {int(target)}"
    if len(observed)>0: cmd+=f" -y {observed}"
    cmd += f" --timeout={timeout} {filename}"
    cmd += f" --log={logfile}"if logfile is not None else ""
    cmd= f"java {vm_flags} -cp {conf.jar_file} {javafile} {cmd}"
    logging.info("command = "+cmd)

    output = exec_bash(cmd)
    
    exec(output[-1])
    return locals()["results"]


In [6]:
def relative_path(path):
    return path.replace(str(conf.prj_path), "./").replace("//","/")

relative_path(conf.model_folder)

'./networks/'

In [7]:
def get_model_path(vertex, kind, nodes, max_degree, max_indegree, max_values, num_vert, idx, **kwargs):
    p  = f""
    p += f"{relative_path(conf.model_folder)}vmodel/vmodel" if vertex else f"{relative_path(conf.model_folder)}hmodel/hmodel"
    p += f"-{kind}_n{nodes}_mID{max_indegree}_mD{max_degree}_mV{max_values}_nV{num_vert}-{idx}.uai"
    return p

def read_query(filename, query_type, field, **kwargs):
    p = filename.replace("vmodel/vmodel", query_type)\
                .replace("hmodel/hmodel", query_type)\
                .replace(relative_path(conf.model_folder), conf.query_folder)\
                .replace(".uai",".csv")
    out = pd.read_csv(p)[field].values[0]
    if pd.isnull(out):
        return ""
    return out


In [8]:
# single_experiemnt

def run(params, start=0, end=None):

    global logfile, strtime, count
    
    logfile, strtime = setup()
    print(logfile)
            
    end = end or len(params)
    params = params[start:end]

    count = 0

    def single_experiment(**t):
        global count, logfile

        logging.info((f"Starting experiment {count} out of {len(params)-1}"))
        count += 1
        
        output = t.copy()
        args = t.copy()
        args["filename"] = args["filename"].replace("./", str(conf.prj_path))
        
        # run query
        res = run_crema(**args, logfile=logfile, timeout=60*10, vm_flags="-Xmx64g")
        
        # get results of interest
        output["interval_result"] = " ".join([f"{p}" for p in res["interval_result"]]) if len(res["err_msg"])==0 else ""
        output["time"] = res["time"] if len(res["err_msg"])==0 else None

        
        # save results for this experiment
        csvfile = f"{conf.res_folder}tmp/{strtime}.csv"
        logging.info(f"Results saved to {csvfile}")
        df = pd.DataFrame([output])
        if os.path.exists(csvfile):
            df.to_csv(csvfile, mode='a', header=False, index=False)
        else:
            df.to_csv(csvfile, index=False)
              
        return output


    df = pd.DataFrame(expd.Experiment(functions=[single_experiment]).run(params))
    
    return df





def build_params(**params_dict):
    func_params = dict(
        filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
        label = lambda t: t.filename.split("/")[-1].split(".uai")[0].replace("model-","").replace("sing","s").replace("mult","m"),
        target = lambda t : read_query(**{**t, **dict(field="target")}),
        observed = lambda t : str(read_query(**{**t, **dict(field="observed")})),
        barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
    )
    transformations = [lambda df: df.drop(columns=["idx"]),
                       lambda df: df[["label"]+[c for c in df.columns if c != "label"]]]
    return expd.generate_input(params_dict, func_params, transformations)



In [30]:
# exact results
params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve"],
    kind = ["sing", "mult"],
    query_type = ["marg", "cond"]
)
params = build_params(**params_dict)
params
#run(params)


,label,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren
0,vs_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,cve,sing,marg,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,3,,0
1,vs_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,cve,sing,cond,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,0,3,0
2,vm_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,cve,mult,marg,./networks/vmodel/vmodel-mult_n4_mID2_mD6_mV4_...,2,,0
3,vm_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,cve,mult,cond,./networks/vmodel/vmodel-mult_n4_mID2_mD6_mV4_...,0,2,0
4,vs_n5_mID2_mD6_mV4_nV2-1,2,6,2,4,5,cve,sing,marg,./networks/vmodel/vmodel-sing_n5_mID2_mD6_mV4_...,4,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,vm_n9_mID6_mD6_mV4_nV6-3,6,6,6,4,9,cve,mult,cond,./networks/vmodel/vmodel-mult_n9_mID6_mD6_mV4_...,3,8 5 1,0
752,vs_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,cve,sing,marg,./networks/vmodel/vmodel-sing_n10_mID6_mD6_mV4...,9,,0
753,vs_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,cve,sing,cond,./networks/vmodel/vmodel-sing_n10_mID6_mD6_mV4...,0,9,0
754,vm_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,cve,mult,marg,./networks/vmodel/vmodel-mult_n10_mID6_mD6_mV4...,3,,3


In [9]:
# approxlp results
params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["approxlp"],
    kind = ["sing", "mult"],
    query_type = ["marg", "cond"]
)
params = build_params(**params_dict)
params
#run(params)


,label,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren
0,hs_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,approxlp,sing,marg,./networks/hmodel/hmodel-sing_n4_mID2_mD6_mV4_...,3,,0
1,hs_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,approxlp,sing,cond,./networks/hmodel/hmodel-sing_n4_mID2_mD6_mV4_...,0,3,0
2,hm_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,approxlp,mult,marg,./networks/hmodel/hmodel-mult_n4_mID2_mD6_mV4_...,2,,0
3,hm_n4_mID2_mD6_mV4_nV2-1,2,6,2,4,4,approxlp,mult,cond,./networks/hmodel/hmodel-mult_n4_mID2_mD6_mV4_...,0,2,0
4,hs_n5_mID2_mD6_mV4_nV2-1,2,6,2,4,5,approxlp,sing,marg,./networks/hmodel/hmodel-sing_n5_mID2_mD6_mV4_...,4,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,hm_n9_mID6_mD6_mV4_nV6-3,6,6,6,4,9,approxlp,mult,cond,./networks/hmodel/hmodel-mult_n9_mID6_mD6_mV4_...,3,8 5 1,0
752,hs_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,approxlp,sing,marg,./networks/hmodel/hmodel-sing_n10_mID6_mD6_mV4...,9,,0
753,hs_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,approxlp,sing,cond,./networks/hmodel/hmodel-sing_n10_mID6_mD6_mV4...,0,9,0
754,hm_n10_mID6_mD6_mV4_nV6-3,6,6,6,4,10,approxlp,mult,marg,./networks/hmodel/hmodel-mult_n10_mID6_mD6_mV4...,3,,3


In [ ]:
# reduced ch results
params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve_ch10","cve_ch5"],
    kind = ["sing", "mult"],
    query_type = ["marg", "cond"]
)
params = build_params(**params_dict)
params
run(params)

/root/dev/2021-SIPTA-crema/logs/20210221_182702.log


In [12]:
from pathlib import Path
res_tmp = Path(conf.res_folder+"/tmp/")
last = sorted(os.listdir(res_tmp))[-1]


last_res = pd.read_csv(os.path.join(res_tmp, last))
last
#new_params = res[res["interval_result"].isnull()].drop(columns=["interval_result"]).reset_index(drop=True)
#new_params

'20210219_152158.csv'

In [ ]:

res = pd.read_csv(os.path.join(res_tmp, last))
new_params = res[res["interval_result"].isnull()].drop(columns=["interval_result"]).reset_index(drop=True)
new_params

In [ ]:
run(params = new_params)

In [ ]:
pd.DataFrame([output])

In [ ]:


!java -Xmx64g -cp /root/dev/2021-SIPTA-crema/target/experiments.jar /root/dev/2021-SIPTA-crema/src/main/java/ch/idsia/RunCrema.java -t --method=cve -r 1 -w 0 -x 0 -y 8.0 --timeout=1600 --log=/root/dev/2021-SIPTA-crema/logs/20210219_110334.log /root/dev/2021-SIPTA-crema/networks/vmodel/vmodel-sing_n9_mID4_mD6_mV4_nV2-1.uai 



In [ ]:
func_params = dict(
    filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
    #target = lambda t : read_query(**{**t, **dict(field="target")}),
    observed = lambda t : str(read_query(**{**t, **dict(field="observed")})),
    #barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
)
transformations = [lambda df: df.drop(columns=["idx"])]
params = expd.generate_input(params_dict, func_params, transformations)



In [ ]:
params.apply(lambda t: len(str(t["observed"]))>2, axis=1).sum()


In [ ]:
params

In [ ]:

def read_query(filename, query_type, field, **kwargs):
    p = filename.replace("vmodel/vmodel", query_type)\
                .replace("vmodel/vmodel", query_type)\
                .replace(relative_path(conf.model_folder), conf.query_folder)\
                .replace(".uai",".csv")
    
    out = pd.read_csv(p)[field].values[0]

    if pd.isnull(out):
        return ""
    return out

filename = f"./networks/vmodel/vmodel-mult_n10_mID2_mD6_mV4_nV2-1.uai"
field = "observed"
query_type = "cond"
read_query(filename, query_type, field)